In [35]:
import pandas as pd
import re
from unidecode import unidecode
from googletrans import Translator
import math
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import Levenshtein as lev

In [36]:
#Lendo os dados
itens = pd.read_csv("items_titles.csv")
itens

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...
...,...
29995,Tênis Vans Old Skool I Love My Vans - Usado - ...
29996,Tênis Feminino Preto Moleca 5296155
29997,Tenis Botinha Com Pelo Via Marte Original Lanç...
29998,Tênis Slip On Feminino Masculino Original Sapa...


In [37]:
#Limpando os textos
itens["ITE_ITEM_TITLE"] = itens["ITE_ITEM_TITLE"].str.upper()
itens["ITE_ITEM_TITLE"] = itens["ITE_ITEM_TITLE"].str.replace(r'[^\w\d\s]', '', regex=True)
itens["ITE_ITEM_TITLE"] = itens["ITE_ITEM_TITLE"].str.replace(r"\s+$", "", regex=True)
itens["ITE_ITEM_TITLE"] = itens["ITE_ITEM_TITLE"].apply(unidecode)
itens

,ITE_ITEM_TITLE
0,TENIS ASCENSION POSH MASCULINO PRETO E VERMELHO
1,TENIS PARA CAMINHADA SUPER LEVINHO SPIDER CORRIDA
2,TENIS FEMININO LE PARC HOCKS BLACKICE ORIGINAL...
3,TENIS OLYMPIKUS ESPORTIVO ACADEMIA NOVA TENDEN...
4,INTELIGENTE LED BICICLETA TAUDA LUZ USB BICICL...
...,...
29995,TENIS VANS OLD SKOOL I LOVE MY VANS USADO FE...
29996,TENIS FEMININO PRETO MOLECA 5296155
29997,TENIS BOTINHA COM PELO VIA MARTE ORIGINAL LANC...
29998,TENIS SLIP ON FEMININO MASCULINO ORIGINAL SAPA...


In [38]:
#Verificando a existencia de duplicados
any(itens.duplicated())

True

In [39]:
#Retirando itens duplicados pois a comparação seria redundandte
itens = itens.drop_duplicates().reset_index(drop=True)
itens

,ITE_ITEM_TITLE
0,TENIS ASCENSION POSH MASCULINO PRETO E VERMELHO
1,TENIS PARA CAMINHADA SUPER LEVINHO SPIDER CORRIDA
2,TENIS FEMININO LE PARC HOCKS BLACKICE ORIGINAL...
3,TENIS OLYMPIKUS ESPORTIVO ACADEMIA NOVA TENDEN...
4,INTELIGENTE LED BICICLETA TAUDA LUZ USB BICICL...
...,...
29939,TENIS VANS OLD SKOOL I LOVE MY VANS USADO FE...
29940,TENIS FEMININO PRETO MOLECA 5296155
29941,TENIS BOTINHA COM PELO VIA MARTE ORIGINAL LANC...
29942,TENIS SLIP ON FEMININO MASCULINO ORIGINAL SAPA...


In [40]:
#Verificando a existencia de duplicados
any(itens.duplicated())

False

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
vectorizer = TfidfVectorizer()

<font size=2> <p style='text-align: justify;'> Estou trabalhando com 10k pois minha máquina não aguentou o processamento. Reduzi a quantidade de dados para poder demosntrar a logica de resolução.

In [43]:
itens_fit = itens['ITE_ITEM_TITLE'][0:10000]

In [44]:
tf_idf_matrix = vectorizer.fit_transform(itens_fit)

In [45]:
tf_idf_matrix.shape

(10000, 8806)

In [46]:
matriz_score = cosine_similarity(tf_idf_matrix)

In [47]:
print(matriz_score)

[[1.         0.00821213 0.00617817 ... 0.04192954 0.01008981 0.00628366]
 [0.00821213 1.         0.00588786 ... 0.00723    0.00961568 0.00598839]
 [0.00617817 0.00588786 1.         ... 0.00543929 0.00723409 0.01941578]
 ...
 [0.04192954 0.00723    0.00543929 ... 1.         0.00888312 0.00553217]
 [0.01008981 0.00961568 0.00723409 ... 0.00888312 1.         0.00735761]
 [0.00628366 0.00598839 0.01941578 ... 0.00553217 0.00735761 1.        ]]


In [48]:
matriz_score_superior = matriz_score.copy()

In [56]:
#Deixa só a diagonal superior
for coluna in range(len(matriz_score_superior)):
    for linha in range(len(matriz_score_superior)):
        if coluna >= linha:
            matriz_score_superior[coluna, linha] = None

In [50]:
print(matriz_score_superior)

[[       nan 0.00821213 0.00617817 ... 0.04192954 0.01008981 0.00628366]
 [       nan        nan 0.00588786 ... 0.00723    0.00961568 0.00598839]
 [       nan        nan        nan ... 0.00543929 0.00723409 0.01941578]
 ...
 [       nan        nan        nan ...        nan 0.00888312 0.00553217]
 [       nan        nan        nan ...        nan        nan 0.00735761]
 [       nan        nan        nan ...        nan        nan        nan]]


In [51]:
matriz_score_superior_sem_diagonal_principal = pd.DataFrame()

In [52]:
melhores_scores = pd.DataFrame()
for linha in range(len(matriz_score_superior)-2):
    melhor_score = sorted(matriz_score_superior[linha])[-2]
    index_melhor_score = list(matriz_score_superior[linha]).index(melhor_score)
    melhores_scores = melhores_scores.append(pd.DataFrame([[itens['ITE_ITEM_TITLE'].iloc[linha], itens['ITE_ITEM_TITLE'].iloc[index_melhor_score], melhor_score]]), ignore_index=True)

In [53]:
melhores_scores.columns = ["ITEM 1", "ITEM 2", "SCORE"]

In [54]:
melhores_scores = melhores_scores.sort_values("SCORE", ascending=False).reset_index(drop=True)

In [55]:
melhores_scores.to_csv("melhores_scores.csv")

In [33]:
melhores_scores

,ITEM 1,ITEM 2,SCORE
0,BICICLETA INFANTIL MENINO ARO 20 BICOLOR ULTRA...,BICICLETA ARO 20 BICOLOR MENINO ULTRA BIKES IN...,1.000000
1,TENIS OLYMPIKUS MASCULINO COMBATE 855,TENIS MASCULINO COMBATE OLYMPIKUS 855,1.000000
2,TENIS VIA MARTE,VIA MARTE,0.984678
3,TENIS ADIDAS RUN 60S 20 FEMININO,TENIS ADIDAS RUN 60S 20 ORIGINAL,0.953727
4,TENIS MIZUNO MASCULINO PROPHECY X,MIZUNO PROPHECY 7,0.938783
...,...,...,...
9993,ONCLOUD X 4344,BICICLETA ARO 29 SPACELINE ORION 21V FREIO A D...,0.000000
9994,GRENDHA,TENIS SAPATENIS MULE PATTERN ABSTRATO MODERNO ...,0.000000
9995,EXCLUSIVA,BICICLETA ARO 29 QUADRO 19 GTI 21V FREIO DISCO...,0.000000
9996,GIUSEPPE ZANOTTI FRANKIE 38BR,SPECIALIZED STUMPJUMPER 29 COMP 11 KILOS,0.000000
